In [1]:
import findspark
findspark.init('/home/yerke/spark-3.3.0-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [4]:
from pyspark.ml.recommendation import ALS

In [6]:
from pyspark.ml.evaluation import RegressionEvaluator

In [8]:
data = spark.read.csv('../course_data/movielens_ratings.csv', inferSchema=True, header=True)

In [9]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [11]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [12]:
training, test = data.randomSplit([.8, .2])

In [13]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [14]:
model = als.fit(training)

In [15]:
predictions = model.transform(test)

In [16]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      0|   1.0|    27|  1.9002224|
|      3|   1.0|    26|  1.9879485|
|      4|   2.0|     1|  1.7781518|
|      0|   1.0|    13|  0.6793296|
|      3|   1.0|    13| 0.77322155|
|      6|   1.0|    13| 0.45004597|
|      1|   1.0|     6|  1.0365182|
|      2|   3.0|     6|  1.7721028|
|      1|   1.0|     3|  0.9566482|
|      1|   1.0|    20| -0.4237549|
|      2|   1.0|    15| 0.72502875|
|      2|   1.0|    17|  3.1954837|
|      3|   1.0|     9|  1.0852026|
|      0|   1.0|     8|  0.7720035|
|      2|   1.0|    23|   4.239903|
|      4|   3.0|    10| 0.35029683|
|      4|   1.0|    29|     1.8997|
|      2|   4.0|    21|   4.366316|
|      3|   1.0|    21|  1.7854056|
|      1|   1.0|    14|-0.90016305|
+-------+------+------+-----------+
only showing top 20 rows



In [17]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [18]:
rmse = evaluator.evaluate(predictions)

In [19]:
print('RMSE')
print(rmse)

RMSE
1.5020185848321408


In [20]:
single_user = test.filter(test['userId']==11).select(['movieId', 'userId'])

In [22]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     10|    11|
|     20|    11|
|     23|    11|
|     36|    11|
|     48|    11|
|     50|    11|
|     61|    11|
|     66|    11|
|     67|    11|
|     71|    11|
|     80|    11|
|     81|    11|
|     86|    11|
|     89|    11|
|     94|    11|
|     99|    11|
+-------+------+



In [24]:
recommendations = model.transform(single_user) 

In [26]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+------------+
|movieId|userId|  prediction|
+-------+------+------------+
|     67|    11|   3.0252037|
|     66|    11|   2.8930955|
|     50|    11|    2.836499|
|     71|    11|   2.8267956|
|     94|    11|   2.4331403|
|     80|    11|   1.9045651|
|     61|    11|   1.4865524|
|     36|    11|   1.1368012|
|     86|    11|  0.91517854|
|     48|    11|  0.86570144|
|     10|    11|   0.8560785|
|     81|    11|   0.8110172|
|     89|    11|   0.4585906|
|     99|    11|-0.097605295|
|     23|    11|   -0.568829|
|     20|    11|  -1.2397834|
+-------+------+------------+

